This notebook contains the code in data scraping.

In [1]:
# Import the dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [76]:
# Setting the url
url = "https://dojos.info/NewJersey/Counties.aspx"

# Creating the response
response = requests.get(url)

print(response)

# Using beautiful soup
soup = bs(response.text,"lxml")

<Response [200]>


In [3]:
# Grabbing the list of counties in New Jersey
state = soup.find('ul', class_ = "olist")
state

<ul class="olist">
<li><h4>A</h4>
<ol class="c"><li><a href="AtlanticCounty/" title="Martial Arts Schools in Atlantic County, NJ">Atlantic County</a> (31)</li>
</ol>
</li>
<li><h4>B</h4>
<ol class="c"><li><a href="BergenCounty/" title="Martial Arts Schools in Bergen County, NJ">Bergen County</a> (126)</li>
<li><a href="BurlingtonCounty/" title="Martial Arts Schools in Burlington County, NJ">Burlington County</a> (44)</li>
</ol>
</li>
<li><h4>C</h4>
<ol class="c"><li><a href="CamdenCounty/" title="Martial Arts Schools in Camden County, NJ">Camden County</a> (55)</li>
<li><a href="CapeMayCounty/" title="Martial Arts Schools in Cape May County, NJ">Cape May County</a> (8)</li>
<li><a href="CumberlandCounty/" title="Martial Arts Schools in Cumberland County, NJ">Cumberland County</a> (15)</li>
</ol>
</li>
<li><h4>E</h4>
<ol class="c"><li><a href="EssexCounty/" title="Martial Arts Schools in Essex County, NJ">Essex County</a> (67)</li>
</ol>
</li>
<li><h4>G</h4>
<ol class="c"><li><a href="G

In [4]:
# Pulling the individual counties
county = state.find_all("a")
county

[<a href="AtlanticCounty/" title="Martial Arts Schools in Atlantic County, NJ">Atlantic County</a>,
 <a href="BergenCounty/" title="Martial Arts Schools in Bergen County, NJ">Bergen County</a>,
 <a href="BurlingtonCounty/" title="Martial Arts Schools in Burlington County, NJ">Burlington County</a>,
 <a href="CamdenCounty/" title="Martial Arts Schools in Camden County, NJ">Camden County</a>,
 <a href="CapeMayCounty/" title="Martial Arts Schools in Cape May County, NJ">Cape May County</a>,
 <a href="CumberlandCounty/" title="Martial Arts Schools in Cumberland County, NJ">Cumberland County</a>,
 <a href="EssexCounty/" title="Martial Arts Schools in Essex County, NJ">Essex County</a>,
 <a href="GloucesterCounty/" title="Martial Arts Schools in Gloucester County, NJ">Gloucester County</a>,
 <a href="HudsonCounty/" title="Martial Arts Schools in Hudson County, NJ">Hudson County</a>,
 <a href="HunterdonCounty/" title="Martial Arts Schools in Hunterdon County, NJ">Hunterdon County</a>,
 <a hre

In [5]:
# Code to create a list of counties.

county_tag = []
state = soup.find('ul', class_ = "olist")
county = state.find_all("a")
for i in range(len(county)):
    county_tag.append(county[i].text)

In [6]:
# To show the county tags.
county_tag

['Atlantic County',
 'Bergen County',
 'Burlington County',
 'Camden County',
 'Cape May County',
 'Cumberland County',
 'Essex County',
 'Gloucester County',
 'Hudson County',
 'Hunterdon County',
 'Mercer County',
 'Middlesex County',
 'Monmouth County',
 'Morris County',
 'Ocean County',
 'Passaic County',
 'Salem County',
 'Somerset County',
 'Sussex County',
 'Union County',
 'Warren County']

In [7]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Kevin\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [9]:
# Visits the url
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [15]:
# Searches the website for the link to the counties
browser.links.find_by_partial_text("County").click()

In [34]:
# Selects the dojo list to click.
ulist = browser.find_by_css('.dojolist')

In [37]:
# Click the the link to find the value.
ulist.find_by_tag('a').click()

In [99]:
# Quits out of the browser.
browser.quit()

In [81]:

url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

In [49]:
address = soup.find('address')
address

<address>
		    2511 Fire Road, Unit 13 A<br/><a href="/NewJersey/EggHarborTownship/">Egg Harbor Township</a>, <a href="/NewJersey/">NJ</a> <a href="/Zip/08234/">08234</a><br/>United States<br/>
</address>

In [52]:
parts = address.find_all("a")

In [54]:
for i in range(3):
    print(parts[i].text)

Egg Harbor Township
NJ
08234


In [64]:
part = soup.find('address', id='text')

In [69]:
soup.find("div", class_ = "phone").text

'\r\n            Phone: 609-415-5014\r\n        '

In [74]:
address.text

'\r\n\t\t    2511 Fire Road, Unit 13 AEgg Harbor Township, NJ 08234United States\n'

In [78]:
address

<address>
		    2511 Fire Road, Unit 13 A<a href="/NewJersey/EggHarborTownship/">Egg Harbor Township</a>, <a href="/NewJersey/">NJ</a> <a href="/Zip/08234/">08234</a>United States
</address>

In [85]:
new_address = address.get_text("|")

In [87]:
new_address.replace(" ","")

'\r\n\t\t2511FireRoad,Unit13A|EggHarborTownship|,|NJ||08234|UnitedStates|\n'

In [90]:
first = new_address.split("|")[0]

In [92]:
second = first.replace("\r\n\t\t","")

In [97]:
third = second.split()
fourth = " ".join(third)
fourth

'2511 Fire Road, Unit 13 A'